In [ ]:
import mxnet as mx
from mxnet import gluon

from matplotlib import pyplot as plt
import numpy as np
import cv2

from cnnviz import cnnviz

In [ ]:
import os
from mxnet.gluon.model_zoo import model_store

from mxnet.initializer import Xavier
from mxnet.gluon.nn import MaxPool2D, Flatten, Dense, Dropout, BatchNorm
from cnnviz.layers import Activation, Conv2D

class VGG(mx.gluon.HybridBlock):
    def __init__(self, layers, filters, classes=1000, batch_norm=False, **kwargs):
        super(VGG, self).__init__(**kwargs)
        assert len(layers) == len(filters)
        with self.name_scope():
            self.features = self._make_features(layers, filters, batch_norm)
            self.features.add(Dense(4096, activation='relu',
                                       weight_initializer='normal',
                                       bias_initializer='zeros'))
            self.features.add(Dropout(rate=0.5))
            self.features.add(Dense(4096, activation='relu',
                                       weight_initializer='normal',
                                       bias_initializer='zeros'))
            self.features.add(Dropout(rate=0.5))
            self.output = Dense(classes,
                                   weight_initializer='normal',
                                   bias_initializer='zeros')

    def _make_features(self, layers, filters, batch_norm):
        featurizer = mx.gluon.nn.HybridSequential(prefix='')
        for i, num in enumerate(layers):
            for _ in range(num):
                featurizer.add(Conv2D(filters[i], kernel_size=3, padding=1,
                                         weight_initializer=Xavier(rnd_type='gaussian',
                                                                   factor_type='out',
                                                                   magnitude=2),
                                         bias_initializer='zeros'))
                if batch_norm:
                    featurizer.add(BatchNorm())
                featurizer.add(Activation('relu'))
            featurizer.add(MaxPool2D(strides=2))
        return featurizer

    def hybrid_forward(self, F, x):
        x = self.features(x)
        x = self.output(x)
        return x
    
vgg_spec = {11: ([1, 1, 2, 2, 2], [64, 128, 256, 512, 512]),
            13: ([2, 2, 2, 2, 2], [64, 128, 256, 512, 512]),
            16: ([2, 2, 3, 3, 3], [64, 128, 256, 512, 512]),
            19: ([2, 2, 4, 4, 4], [64, 128, 256, 512, 512])}

def get_vgg(num_layers, pretrained=False, ctx=mx.cpu(),
            root=os.path.join('~', '.mxnet', 'models'), **kwargs):
    layers, filters = vgg_spec[num_layers]
    net = VGG(layers, filters, **kwargs)
    if pretrained:
        from mxnet.gluon.model_zoo.model_store import get_model_file
        batch_norm_suffix = '_bn' if kwargs.get('batch_norm') else ''
        net.load_params(get_model_file('vgg%d%s'%(num_layers, batch_norm_suffix),
                                       root=root), ctx=ctx)
    return net

def vgg16(**kwargs):
    return get_vgg(16, **kwargs)

In [ ]:
image_sz = (224, 224)

def preprocess(data):
    data = mx.image.imresize(data, image_sz[0], image_sz[1])
    data = data.astype(np.float32)
    data = data/255
    data = mx.image.color_normalize(data,
                                    mean=mx.nd.array([0.485, 0.456, 0.406]),
                                    std=mx.nd.array([0.229, 0.224, 0.225]))
    data = mx.nd.transpose(data, (2,0,1))
    return data

network = vgg16(pretrained=True, ctx=mx.cpu())

In [ ]:
def inference(img_path):
    with open(img_path, 'rb') as fp:
        str_image = fp.read()

    #print(str_image)
    image = mx.img.imdecode(str_image)
    image = preprocess(image)
    image = image.expand_dims(axis=0)
    #image = image.as_in_context(mx.gpu())

    out = network(image)
    return out.argmax(axis=1).asnumpy()[0].astype(int)   

In [ ]:
# for i in range(10000):
#     inference("img/snake.jpg")

In [ ]:
synset = []
with open('synset.txt', 'r') as f:
    clsnames = [l.rstrip().split(' ', 1)[1].split(',')[0] for l in f]
    
with open('synset.txt', 'r') as f:
    clsnums = [l.rstrip().split(' ', 1)[0] for l in f]

clsindices = {}
for i, cn in enumerate(clsnums):
    clsindices[cn] = i

In [ ]:
clsindices

In [ ]:
base_dir = '/ssd2/base'
train_dir = "%s/train" % base_dir

for d in os.listdir(train_dir):
    img_dir = "%s/%s/images" % (train_dir, d)
    expected_cls_indx = clsindices[d]
    for img_name in os.listdir(img_dir):
        img_path = "%s/%s" % (img_dir, img_name)
        #print(img_path)
        infer_cls_indx = inference(img_path)
        if expected_cls_indx != infer_cls_indx:
            print("%s: Expected: %s, Inferred: %s" % (img_path, clsnames[expected_cls_indx], clsnames[infer_cls_indx]))
